# Reading VEX Stats

This notebook is a demonstration of using the RobotEvents.com API to retrieve and analyse robotics competition data.

[API Documentation](https://www.robotevents.com/api/v2)

[Terms of use for robotevents.com](https://www.vexrobotics.com/terms-of-use)

## Setup

This first cell will get everything set up, including our Data Dunkers API token. You may need to set up your own token by [requesting access](https://www.robotevents.com/api/v2/accessRequest/create) and then [creating a new token](https://www.robotevents.com/api/v2/tokens).

In [ ]:
token = ''

import piplite, pyodide_http, pyodide
await piplite.install(['pandas', 'plotly', 'nbformat', 'statsmodels'])
pyodide_http.patch_all()
import requests
import pandas as pd
import plotly.express as px
#df = pd.read_csv(pyodide.http.open_url('https://docs.google.com/spreadsheets/d/1KeUkR2qpsaFdU2Lvks79vju22fJNzFB-iGfGUxmPQHI/export?format=csv'))

headers = {'Authorization': f'Bearer {token}'}
print('setup completed')

## Teams

Now that we have everything set up, we can get data about an individual team.

In [ ]:
team_number = '7410X'

team_data = requests.get(f'https://www.robotevents.com/api/v2/teams?number={team_number}', headers=headers).json()
team_data

In [ ]:
team_data['data'][0]['organization']

We can also get event data.

In [ ]:
event_code = 'RE-V5RC-24-6445'

event_data = requests.get(f'https://www.robotevents.com/api/v2/events?code={event_code}', headers=headers).json()
event_data

We need to do some parsing to flatten that [JSON](https://en.wikipedia.org/wiki/JSON) so we can create a [pandas dataframe](https://en.wikipedia.org/wiki/Pandas_(software)#DataFrames).

In [ ]:
records = []
for e in event_data['data']:
  record = {
      'id': e.get('id'),
      'sku': e.get('sku'),
      'name': e.get('name'),
      'start': e.get('start'),
      'end': e.get('end'),
      'season_id': e.get('season', {}).get('id'),
      'season_name': e.get('season', {}).get('name'),
      'program_id': e.get('program', {}).get('id'),
      'program_name': e.get('program', {}).get('name'),
      'program_code': e.get('program', {}).get('code'),
      'venue': e.get('location', {}).get('venue'),
      'city': e.get('location', {}).get('city'),
      'region': e.get('location', {}).get('region'),
      'country': e.get('location', {}).get('country'),
      'lat': e.get('location', {}).get('coordinates', {}).get('lat'),
      'lon': e.get('location', {}).get('coordinates', {}).get('lon'),
      'level': e.get('level'),
      'ongoing': e.get('ongoing'),
      'awards_finalized': e.get('awards_finalized'),
      'divisions': ', '.join([d.get('name', '') for d in e.get('divisions', [])]),  # join division names if there are multiple
    }
  records.append(record)
df = pd.DataFrame(records)
df

Alternatively, we could use `json_normalize` on the columns that contain dictionaries or lists. Remove the `'''` lines to run this cell if you'd like, but your resulting dataframe may be messier.

In [ ]:
'''
df = pd.DataFrame(event_data['data'])

for column in df.columns:
  if type(df[column][0]) == dict and column != 'locations': # we'll skip flatening the locations column
    temp_df = pd.json_normalize(df[column]).add_prefix(column + '_')
    df = pd.concat([df, temp_df], axis=1)
    df = df.drop(columns=[column])
  elif type(df[column][0]) == list:
    temp_df = pd.json_normalize(df[column]).apply(lambda x: x[0])
    temp_df = temp_df.add_prefix(column + '_')
    df = pd.concat([df, temp_df], axis=1)
    df = df.drop(columns=[column])
df
'''

Let's have a look at the column names.

In [ ]:
df.columns

We can visualize some data with [Plotly Express](https://plotly.com/python/plotly-express/).

In [ ]:
px.scatter(df, x='lon', y='lat', color='season_name', title='VEX Events')

## Seasons

We can also get data about the competition seaso and convert it into a flattened dataframe.s.

In [ ]:
seasons = requests.get('https://www.robotevents.com/api/v2/seasons?per_page=250', headers=headers).json()
temp_df = pd.DataFrame(seasons['data'])
seasons_data = pd.concat([temp_df, pd.json_normalize(temp_df['program']).add_prefix('program' + '_')], axis=1).drop(columns=['program'])
seasons_data

Let's look at just V5 (`'program_id' == 1`).

In [ ]:
seasons_data[seasons_data['program_id'] == 1]

We can use the API again to pull data about events from last season (`'id' == 190`).

In [ ]:
id = 190

events = []
page = 1

while True:
  r = requests.get(f'https://www.robotevents.com/api/v2/seasons/{id}/events?page={page}&per_page=250', headers=headers).json()
  events.extend(r['data'])
  if r['meta']['next_page_url']:
    page += 1
  else:
    break
events_data = pd.DataFrame(events)
events_data

And we can expand some of the columns that contain dictionaries.

In [ ]:
events_data = pd.DataFrame(events)

columns = ['season', 'program', 'location']
for column in columns:
  temp_df = pd.json_normalize(events_data[column]).add_prefix(column + '_')
  events_data = pd.concat([events_data, temp_df], axis=1)
events_data = events_data.drop(columns=columns)

events_data

Hopefully this is a good start for data science with [RobotEvents.com](https://RobotEvents.com) data.